In [ ]:
%pip install gradio
%pip install --upgrade --quiet  langchain-google-genai
%pip install langchain-anthropic
%pip install langchain-groq
%pip install langchain
%pip install pypdf
%pip install -q langchain faiss-cpu sentence-transformers==2.2.2 InstructorEmbedding pypdf
%pip install langchain PyPDF2 faiss-cpu huggingface-hub pandas
%pip install -U langchain-community
%pip install -U langchain-community faiss-cpu

In [ ]:
import getpass
import os
from langchain_google_genai import GoogleGenerativeAI
from langchain_anthropic import ChatAnthropic
from langchain_groq import ChatGroq
from langchain.memory import ConversationBufferMemory
from langchain.agents import  initialize_agent
from pypdf import PdfReader
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.document_loaders import TextLoader
from pypdf import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferWindowMemory
import pickle
import gradio as gr

In [ ]:
def read_pdf(file):
    document = ""

    reader = PdfReader(file)
    for page in reader.pages:
        document += page.extract_text()

    return document

# Document Splitting
def split_doc(document, chunk_size, chunk_overlap):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    split = splitter.split_text(document)
    split = splitter.create_documents(split)
    return split
def prepare_rag_llm(
    token, llm_model, embeddings_name, temperature, max_length, split, new_vs_name
):
    embeddings = GoogleGenerativeAIEmbeddings(model=embeddings_name,google_api_key=token)

    db = FAISS.from_documents(split, embeddings)
    db.save_local("vector_store/" + new_vs_name)

    loaded_db = FAISS.load_local(
        f"vector_store/{new_vs_name}", embeddings, allow_dangerous_deserialization=True
    )

    llm = GoogleGenerativeAI(
        model=llm_model,
        google_api_key=token,
        temperature=temperature,
        max_length=max_length
    )

    memory = ConversationBufferWindowMemory(
        k=2,
        memory_key="chat_history",
        output_key="answer",
        return_messages=True,
    )

    qa_conversation = ConversationalRetrievalChain.from_llm(
        llm=llm,
        chain_type="stuff",
        retriever=loaded_db.as_retriever(),
        return_source_documents=True,
        memory=memory,
    )

    return qa_conversation
pdf_file_path = "/content/drive/MyDrive/FINAL_SCRAPED_DATA.pdf"
document = read_pdf(pdf_file_path)
split = split_doc(document, chunk_size=1000, chunk_overlap=200)
llm_model='gemini-1.0-pro'
instruct_embeddings = "models/embedding-001"
vector_store_list = "insti_vector_store"
temperature = 1.0
max_length = 512
token  = ""

qa_conversation = prepare_rag_llm(token, llm_model, instruct_embeddings, temperature, max_length, split, new_vs_name="insti_vector_store")
def chatbot_interface(user_input, history):
    history = history or []
    response = qa_conversation({"question": user_input, "chat_history": history})
    history.append((user_input, response["answer"]))
    return history, history
with gr.Blocks() as demo:
    # Add title
    gr.Markdown("# InstiGPT")

    # Define the chatbot and state
    chatbot = gr.Chatbot()
    state = gr.State([])

    # Layout for text input and button
    with gr.Row():
        txt = gr.Textbox(
            show_label=False,
            placeholder="Enter prompt.",
            lines=1,
            scale=3
        )
        btn = gr.Button("Send")  # Enter button

    # Configure interaction
    txt.submit(chatbot_interface, [txt, state], [chatbot, state])
    btn.click(chatbot_interface, [txt, state], [chatbot, state])

# Launch the interface
demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://8e8a62efcdf0f5c5f2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
